In [1]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import random
import zipfile
import time
import shutil
import zipcode
import seaborn as sns
%matplotlib inline

In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """
    Use of this metric is not recommended; for illustration only. 
    See other regression metrics on sklearn docs:
      http://scikit-learn.org/stable/modules/classes.html#regression-metrics
    Use like any other metric
    >>> y_true = [3, -0.5, 2, 7]; y_pred = [2.5, -0.3, 2, 8]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    Out[]: 24.791666666666668
    """

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
def mape(preds, dtrain): 
    
    labels = dtrain.get_label()
    
    return 'mape',np.mean(np.abs((labels - preds) / labels)) * 100

In [4]:
from sklearn.metrics import make_scorer

In [5]:
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [6]:
train = pd.read_csv('Fr_train.csv', sep=';')
test = pd.read_csv('Fr_test.csv', sep=';')

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
correlation = train.select_dtypes(exclude=['object']).corr()

In [8]:
target = train.prime_tot_ttc
train = train.drop(['prime_tot_ttc'],axis=1)

In [9]:
post_state = []
post_wage = []
post_population = []

for i in train.codepostal.values:
    
    try:
        state = zipcode.isequal(str(i)).state
        post_state.append(state)
    except:
        post_state.append('nothing')

    try:
        wage = int(zipcode.isequal(str(i)).wage)
        post_wage.append(wage)
    except:
        post_wage.append(0)
        
    try:
        pop = int(zipcode.isequal(str(i)).population)
        post_population.append(pop)
    except:
        post_population.append(0)

In [10]:
post_state_test = []
post_wage_test = []
post_population_test = []

for i in test.codepostal.values:
    
    try:
        state = zipcode.isequal(str(i)).state
        post_state_test.append(state)
    except:
        post_state_test.append('no_state')
        
    try:
        wage = int(zipcode.isequal(str(i)).wage)
        post_wage_test.append(wage)
    except:
        post_wage_test.append(0)
        
    try:
        pop = int(zipcode.isequal(str(i)).population)
        post_population_test.append(pop)
    except:
        post_population_test.append(0)

In [11]:
test['state'] = post_state_test
test['wage'] = post_wage_test
test['population'] = post_population_test

In [12]:
test = test.drop(['codepostal'],axis=1)

In [13]:
train['state'] = post_state
train['wage'] = post_wage
train['population'] = post_population

In [14]:
train = train.drop(['codepostal'],axis=1)

In [15]:
train = train.drop(['id'],axis=1)
test_id = test.id
test = test.drop(['id'],axis=1)

In [16]:
train = train.fillna(train.median())

In [17]:
test = test.fillna(test.median())

In [18]:
train_dummy = pd.get_dummies(train)

In [19]:
test_dummy = pd.get_dummies(test)

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import cross_val_score

In [ ]:
score = []
for i in range(3,10,1):
    gb = GradientBoostingRegressor(n_estimators=400, verbose=1, max_depth=i)
    score.append(cross_val_score(gb, train_dummy, target, cv=3, n_jobs=-1,scoring=scorer).mean())
    print(i, score)

      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
         1       12953.2699           13.61m
      Iter       Train Loss   Remaining Time 
         1       12811.8879           12.86m
         2       11743.1195           13.56m
         1       12858.4264           13.09m
         2       11582.6015           12.79m
         3       10661.4085           13.42m
         2       11614.0179           13.08m
         3       10552.4292           12.67m
         4        9792.9024           13.31m
         3       10560.1436           12.96m
         4        9694.7332           12.68m
         5        9028.9715           13.16m
         4        9682.4202           12.98m
         5        8927.6022           12.54m
         6        8388.0141           13.06m
         5        8925.0966           12.84m
         6        8294.9674           12.50m
         7        7832.5701           13.03m
         6        8286.7187           12.74m
       

Process PoolWorker-64:
Process PoolWorker-60:
Process PoolWorker-61:
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-62:
Process PoolWorker-63:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/ubuntu/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
    self.run()
  File "/home/ubuntu/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/ubuntu/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/ubuntu/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/home/ubuntu/anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run


In [42]:
gb = GradientBoostingRegressor(n_estimators=10)
gb.fit(train.select_dtypes(exclude=['object']), target)

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [43]:
prediction = gb.predict(test.select_dtypes(exclude=['object']))

In [44]:
prediction

array([ 238.84167098,  238.84167098,  238.84167098, ...,  238.84167098,
        238.84167098,  238.84167098])

In [48]:
result = pd.DataFrame([test.id.values,prediction])
result = result.transpose()
cols = ['ID','COTIS']
result.columns = cols
result.ID = result.ID.astype(int)

In [49]:
result.to_csv('result.csv',sep=';',index_label=False, index=False)